# 0.0 Import Libraries

In [ ]:
pip install spectral mat73  einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import math

from einops import rearrange
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from scipy import io
import torch.utils.data
import scipy.io as sio
import mat73
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1.0 Upload Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! ls '/content/drive/MyDrive/A02_RemoteSensingData/UHS_2013_DFTC/'

 2013_DFTC					   best_model_p7_weights.pth
 2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat   best_model_p9_weights.pth
 2013_IEEE_GRSS_DF_Contest_LiDAR.mat		   best_model.pth
 Autoencodermodel.pth				   best_model_weights.pth
 Autoencodermodel_uh2013_20Ksample.pth		   BS_RLFCC_UH2013.ipynb
 Autoencodermodel_uh2013_adam_20Ksample.pth	   cae_model_state_batchscore.pth
 Autoencodermodel_uh2013_adamp13_20Ksample.pth	   cae_model_state.pth
 Autoencodermodel_uh2013_adamp9_20Ksample.pth	  'Contrastive Best model'
 Autoencodermodel_uh2013.pth			   cross_modal_autoencoder_best.pth
 Autoencodermodel_uh2013_rms_20Ksample.pth	   cross_modal_autoencoder.pth
 Autoencodermodel_uh2013_sgdp13_20Ksample.pth	   DFTC2013_Fusion_Model.h5
 Autoencodermodel_uh2013_sgdp3_20Ksample.pth	   GRSS2013.mat
 Autoencodermodel_uh2013_sgdp5_20Ksample.pth	   __MACOSX
 Autoencodermodel_uh2013_sgdp7_20Ksample.pth	   model_path.pth
 Autoencodermodel_uh2013_sgdp9v1_20Ksample.pth	   model.pth
 best_contrastive_model.pth			

In [ ]:
# Define the path
path='/content/drive/MyDrive/A02_RemoteSensingData/UHS_2013_DFTC/'

In [ ]:
# 2.1 Loads Data
# Load hyperpsectral data
hsi_2013_data=sio.loadmat(path+'2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
print('hsi_2013_data shape:', hsi_2013_data.shape)

# Loader Lidar  data
import mat73
lidar_2013_data = sio.loadmat(path+'2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

print('Lidar_2013_data shape:', lidar_2013_data.shape)

#Load ground truth labels
gt_2013_data=sio.loadmat(path+'GRSS2013.mat')['name']
print('gt_2013_data.shape:', gt_2013_data.shape)

hsi_2013_data shape: (349, 1905, 144)
Lidar_2013_data shape: (349, 1905, 1)
gt_2013_data.shape: (349, 1905)


# 2.0 Data Preprocessing & Dataloader Preparation

In [ ]:
# 2.1 Define the class information
class_info = [(1, "Healthy grass", 'training_sample', 198, 'test_sample', 1053,  'total', 1251),
    (2, "Stressed grass",'training_sample', 190, 'test_sample', 1064,  'total', 1254),
    (3, "Synthetic grass", 'training_sample', 192, 'test_sample', 505,  'total', 697),
    (4, "Trees", 'training_sample', 188, 'test_sample', 1058,  'total', 1244),
    (5, "Soil",'training_sample', 186, 'test_sample', 1056,  'total', 1242),
    (6, "Water", 'training_sample', 182, 'test_sample', 141,  'total', 325),
    (7, "Residential", 'training_sample', 196, 'test_sample', 1072,  'total', 1268),
    (8, "Commercial", 'training_sample', 191, 'test_sample', 1053,  'total', 1244),
    (9, "Road", 'training_sample', 193, 'test_sample', 1059,  'total', 1252),
    (10, "Highway", 'training_sample', 191, 'test_sample', 1036,  'total', 1227),
    (11, "Railway", 'training_sample', 181, 'test_sample', 1054,  'total', 1235),
    (12, "Parking lot 1", 'training_sample', 192, 'test_sample', 1041,  'total', 1233),
    (13, "Parking lot 2", 'training_sample', 184, 'test_sample',285,  'total', 469),
    (14, "Tennis court",'training_sample', 181, 'test_sample', 247,  'total', 428),
    (15, "Running track", 'training_sample', 187, 'test_sample', 473,  'total', 660)]

# Create a dictionary to store class number, class name, and class samples
class_dict = {class_number: {"class_name": class_name,
                             'training_sample': training_sample,
                             'test_sample': test_sample,
                             "total_samples": total}
              for class_number, class_name, _, training_sample, _, test_sample, _, total in class_info}

print(class_dict)


{1: {'class_name': 'Healthy grass', 'training_sample': 198, 'test_sample': 1053, 'total_samples': 1251}, 2: {'class_name': 'Stressed grass', 'training_sample': 190, 'test_sample': 1064, 'total_samples': 1254}, 3: {'class_name': 'Synthetic grass', 'training_sample': 192, 'test_sample': 505, 'total_samples': 697}, 4: {'class_name': 'Trees', 'training_sample': 188, 'test_sample': 1058, 'total_samples': 1244}, 5: {'class_name': 'Soil', 'training_sample': 186, 'test_sample': 1056, 'total_samples': 1242}, 6: {'class_name': 'Water', 'training_sample': 182, 'test_sample': 141, 'total_samples': 325}, 7: {'class_name': 'Residential', 'training_sample': 196, 'test_sample': 1072, 'total_samples': 1268}, 8: {'class_name': 'Commercial', 'training_sample': 191, 'test_sample': 1053, 'total_samples': 1244}, 9: {'class_name': 'Road', 'training_sample': 193, 'test_sample': 1059, 'total_samples': 1252}, 10: {'class_name': 'Highway', 'training_sample': 191, 'test_sample': 1036, 'total_samples': 1227}, 11: 

### 2.1  Samples Extraction

In [ ]:
# 2.2 Samples Extraction

# # Create a mask with all class labels
# mask = np.copy(gt_2013_data)

# # Set the background class to 0
# mask[mask == 0] = 0

# Define patch size and stride
patch_size = 9
stride = 1

# Create an empty list to store patches and labels
hsi_samples = []
lidar_samples = []
labels = []

# Initialize a dictionary to store class count
class_count = {i: 0 for i in class_dict.keys()}

# Function to check if all classes have the required number of samples
def all_classes_completed(class_count, class_dict):
    return all(class_count[class_num] == class_dict[class_num]["total_samples"] for class_num in class_dict.keys())

while not all_classes_completed(class_count, class_dict):
    # Loop through the ground truth data
    for label in class_dict.keys():
        # Get the coordinates of the ground truth pixels
        #coords = np.argwhere((gt_2013_data == label) & (mask > 0))
        coords = np.argwhere(gt_2013_data == label)

        # Shuffle the coordinates to randomize the patch extraction
        np.random.shuffle(coords)

        for coord in coords:
            i, j = coord
            # Calculate the patch indices
            i_start, i_end = i - patch_size // 2, i + patch_size // 2 + 1
            j_start, j_end = j - patch_size // 2, j + patch_size // 2 + 1

            # Check if the indices are within the bounds of the HSI data
            if i_start >= 0 and i_end <= hsi_2013_data.shape[0] and j_start >= 0 and j_end <= hsi_2013_data.shape[1]:
                # Extract the patch
                hsi_patch = hsi_2013_data[i_start:i_end, j_start:j_end, :]

                # Extract the LiDAR patch
                lidar_patch = lidar_2013_data[i_start:i_end, j_start:j_end, :]

                # If the class count is less than the required samples
                if class_count[label] < class_dict[label]["total_samples"]:
                    # Append the patch and its label to the list
                    hsi_samples.append(hsi_patch)
                    lidar_samples.append(lidar_patch)
                    labels.append(label)
                    class_count[label] += 1

                    # If all classes have the required number of samples, exit the loop
                    if all_classes_completed(class_count, class_dict):
                        break

# Convert the list of patches and labels into arrays
hsi_samples = np.array(hsi_samples)
lidar_samples = np.array(lidar_samples)
labels = np.array(labels)
print('hsi_samples shape:', hsi_samples.shape)
print('lidar_samples shape:', lidar_samples.shape)
print('labels shape:', labels.shape)

hsi_samples shape: (15029, 9, 9, 144)
lidar_samples shape: (15029, 9, 9, 1)
labels shape: (15029,)


### 2.2 Training samples extraction

In [ ]:
#Avoid overlap of train and test
# Extracting training samples
hsi_training_samples, lidar_training_samples, training_labels = [], [], []
used_indices = []  # To keep track of indices already taken for training samples

for label, class_data in class_dict.items():
    # Get indices of the current class
    class_indices = np.where(labels == label)[0]

    # Randomly shuffle the indices
    np.random.shuffle(class_indices)

    # Take the required number of training samples
    train_indices = class_indices[:class_data["training_sample"]]
    used_indices.extend(train_indices)  # Add these to the used_indices list

    # Append training samples
    hsi_training_samples.extend(hsi_samples[train_indices])
    lidar_training_samples.extend(lidar_samples[train_indices])
    training_labels.extend(labels[train_indices])

# Extracting test samples
hsi_test_samples, lidar_test_samples, test_labels = [], [], []

for label, class_data in class_dict.items():
    class_indices = np.where(labels == label)[0]

    # Exclude indices which were used for training
    test_indices = np.setdiff1d(class_indices, used_indices)

    # Append test samples
    hsi_test_samples.extend(hsi_samples[test_indices])
    lidar_test_samples.extend(lidar_samples[test_indices])
    test_labels.extend(labels[test_indices])

# Convert lists back to numpy arrays
hsi_training_samples = np.array(hsi_training_samples)
lidar_training_samples = np.array(lidar_training_samples)
training_labels = np.array(training_labels)

hsi_test_samples = np.array(hsi_test_samples)
lidar_test_samples = np.array(lidar_test_samples)
test_labels = np.array(test_labels)

# Print shapes to verify
print('hsi_training_samples shape:', hsi_training_samples.shape)
print('lidar_training_samples shape:', lidar_training_samples.shape)
print('training_labels shape:', training_labels.shape)

print('hsi_test_samples shape:', hsi_test_samples.shape)
print('lidar_test_samples shape:', lidar_test_samples.shape)
print('test_labels shape:', test_labels.shape)


hsi_training_samples shape: (2832, 9, 9, 144)
lidar_training_samples shape: (2832, 9, 9, 1)
training_labels shape: (2832,)
hsi_test_samples shape: (12197, 9, 9, 144)
lidar_test_samples shape: (12197, 9, 9, 1)
test_labels shape: (12197,)


### 2.2 Augmented samples

In [ ]:
import numpy as np
from scipy.ndimage import rotate

def augment_training_data(hsi_training_data, lidar_training_data, training_labels, rotations=[45, 90, 135], flip_up_down=True, flip_left_right=True):
    augmented_hsi = []
    augmented_lidar = []
    augmented_labels = []

    for hsi, lidar, label in zip(hsi_training_data, lidar_training_data, training_labels):
        # Original data
        augmented_hsi.append(hsi)
        augmented_lidar.append(lidar)
        augmented_labels.append(label)

        # Rotations
        for angle in rotations:
            hsi_rotated = rotate(hsi, angle, axes=(0, 1), reshape=False, mode='nearest')
            lidar_rotated = rotate(lidar, angle, axes=(0, 1), reshape=False, mode='nearest')

            augmented_hsi.append(hsi_rotated)
            augmented_lidar.append(lidar_rotated)
            augmented_labels.append(label)

        # Flip up-down
        if flip_up_down:
            hsi_flipped_ud = np.flipud(hsi)
            lidar_flipped_ud = np.flipud(lidar)

            augmented_hsi.append(hsi_flipped_ud)
            augmented_lidar.append(lidar_flipped_ud)
            augmented_labels.append(label)

        # Flip left-right
        if flip_left_right:
            hsi_flipped_lr = np.fliplr(hsi)
            lidar_flipped_lr = np.fliplr(lidar)

            augmented_hsi.append(hsi_flipped_lr)
            augmented_lidar.append(lidar_flipped_lr)
            augmented_labels.append(label)

    return np.array(augmented_hsi), np.array(augmented_lidar), np.array(augmented_labels)

# Augmenting the training samples
augmented_hsi_training_samples, augmented_lidar_training_samples, augmented_training_labels = augment_training_data(hsi_training_samples, lidar_training_samples, training_labels)

# Print shapes to verify the augmented training data
print('Augmented HSI training samples shape:', augmented_hsi_training_samples.shape)
print('Augmented LiDAR training samples shape:', augmented_lidar_training_samples.shape)
print('Augmented training labels shape:', augmented_training_labels.shape)

Augmented HSI training samples shape: (16992, 9, 9, 144)
Augmented LiDAR training samples shape: (16992, 9, 9, 1)
Augmented training labels shape: (16992,)


In [ ]:
hsi_train=augmented_hsi_training_samples
lidar_train=augmented_lidar_training_samples
y_train=augmented_training_labels
print('hsi_train_samples shape:', hsi_train.shape)
print('lidar_train_samples shape:', lidar_train.shape)
print('train_labels shape:', y_train.shape)
hsi_test=hsi_test_samples
lidar_test=lidar_test_samples
y_test=test_labels
print('hsi_test_samples shape:', hsi_test.shape)
print('lidar_test_samples shape:', lidar_test.shape)
print('y_test shape:', y_test.shape)

hsi_train_samples shape: (16992, 9, 9, 144)
lidar_train_samples shape: (16992, 9, 9, 1)
train_labels shape: (16992,)
hsi_test_samples shape: (12197, 9, 9, 144)
lidar_test_samples shape: (12197, 9, 9, 1)
y_test shape: (12197,)


In [ ]:
from tensorflow.keras.utils import to_categorical

# One hot encoding of labels
# Substract 1 from labels
y_train_adj = augmented_training_labels - 1
y_test_adj = y_test - 1

# One hot encoding of labels
y_train = to_categorical(y_train_adj, num_classes = 15, dtype ="int32")
y_test = to_categorical(y_test_adj, num_classes = 15, dtype ="int32")

print('y_train.shape:',y_train.shape)
print('y_test.shape:',y_test.shape)

y_train.shape: (16992, 15)
y_test.shape: (12197, 15)


#3.0 Model Building

### 3.1 Configuration

In [ ]:
# 3.1 Configuration
class Config:
    def __init__(self,in_channels,num_patches,kernel_size,patch_size,emb_size, dim,depth,heads,dim_head,mlp_dim,num_classes,dropout,pos_emb_size,class_emb_size,stride, ):
        self.in_channels = in_channels
        self.num_patches = num_patches
        self.kernel_size = kernel_size
        self.patch_size = patch_size
        self.emb_size = emb_size
        self.dim = dim
        self.depth = depth
        self.heads = heads
        self.dim_head = dim_head
        self.mlp_dim = mlp_dim
        self.num_classes = num_classes
        self.dropout = dropout
        self.pos_emb_size = pos_emb_size
        self.class_emb_size = class_emb_size
        self.stride = stride


### 3.2 EmbeddingPatches

In [ ]:

class PatchEmbedding(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.proj = nn.Conv2d(
            config.in_channels,
            config.emb_size,
            kernel_size=config.kernel_size,
            stride=config.stride,
        )
        self.pos_embedding = nn.Parameter(torch.zeros(1, config.num_patches + 1, config.emb_size))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, config.emb_size))

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.proj(x)
        x = x.flatten(2).transpose(1, 2)
        x = torch.cat((self.cls_token.expand(B, -1, -1), x), dim=1)
        x = x + self.pos_embedding[:, :x.size(1)]
        return x



### 3.2 Optianl Adding bandoutput

In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, lidar_config, hsi_config):
        super(CrossAttention, self).__init__()

        # Define module parameters
        self.dim_head = lidar_config.dim_head
        self.num_patches=hsi_config.num_patches
        self.num_heads = lidar_config.heads
        self.sqrt_dim_head = math.sqrt(self.dim_head)

        # Define linear layers for transforming Q, K, and V
        self.to_q = nn.Linear(lidar_config.dim, self.dim_head * self.num_heads, bias=False)
        self.to_k = nn.Linear(hsi_config.dim, self.dim_head * self.num_heads, bias=False)
        self.to_v = nn.Linear(lidar_config.dim, self.dim_head * self.num_heads, bias=False)

        self.to_out = nn.Linear( self.num_heads* self.dim_head,self.num_patches )  # added


    def forward(self, lidar, hsi):
        B, N_lidar, _ = lidar.size()
        _, N_hsi, _ = hsi.size()

        outputs = []
        attn_scores = []  # List to store attention scores

       # Iterate over lidar and hsi patches
        for i in range(1, N_lidar):
        #for i in range(N_lidar):

            lidar_patch = lidar[:, i].unsqueeze(1)  # Add a dimension for number of patches
            for j in range(1, N_hsi):
            #for j in range(N_hsi):

                hsi_patch = hsi[:, j].unsqueeze(1)  # Add a dimension for number of patches
                Q = self.to_q(lidar_patch)
                K = self.to_k(hsi_patch)
                V = self.to_v(lidar_patch)

                Q = Q / self.sqrt_dim_head
                attn_weights = F.softmax(Q @ K.transpose(-2, -1), dim=-1)

                attn_output = attn_weights @ V
                attn_score = self.to_out(attn_output)  # added
                outputs.append(attn_output)
                attn_scores.append(attn_score)  # Store the attention scores

         # Concatenate all the outputs
        output = torch.cat(outputs, dim=1)
        attn_scores = torch.cat(attn_scores, dim=1)  # Concatenate all the attention scores

        return output, attn_scores  # Return both output and attention scores


In [ ]:
class CrossAttentionModel(nn.Module):
    def __init__(self, hsi_config, lidar_config):
        super().__init__()
        self.hsi_patch_embedding = PatchEmbedding(hsi_config)
        self.lidar_patch_embedding = PatchEmbedding(lidar_config)
        self.cross_attention = CrossAttention(lidar_config, hsi_config)

    def forward(self, lidar_data, hsi_data):
        # Apply PatchEmbedding
        lidar_emb = self.lidar_patch_embedding(lidar_data)
        hsi_emb = self.hsi_patch_embedding(hsi_data)

        # Apply CrossAttention
        output, attn_scores = self.cross_attention(lidar_emb, hsi_emb)

        return output, attn_scores  # Return both output and attention scores


In [ ]:
#3.1.1 Parameters Setting
# Hsi configuration
hsi_config = Config(
    in_channels=81,  # Each sample covers 144 bands
    num_patches=144,  # 25*1*144 bands are grouped into 3600 groups
    kernel_size=1,  # Adjusted to match new patch size
    patch_size=9,  # Adjusted to match new patch size (5*5, 144/24=6)
    emb_size=128,  # Embedding size, this can be changed
    dim=128,  # Dimension of the transformer, this can be changed
    depth=3,  # Number of transformer layers, this can be changed
    heads=8,  # Number of attention heads, this can be changed
    dim_head=64,  # Dimension of each attention head, this can be changed
    mlp_dim=256,  # Dimension of the MLP layer, this can be changed
    num_classes=15,  # Number of classes, this can be changed
    dropout=0.4,  # Dropout rate, this can be changed
    pos_emb_size=128,  # Position embedding size, this can be changed
    class_emb_size=128,  # Class embedding size, this can be changed
    stride=1  # Stride for the convolution, this can be changed
)


# Lidara configuration
lidar_config = Config(
    in_channels=81,  # lidar group has 1 channels
    num_patches=1,  # 1 band for Lidar
    kernel_size=1,  # Adjusted to match new patch size
    patch_size=9, # Adjusted to match new patch size
    emb_size=128,  # Embedding size, this can be changed
    dim=128,  # Dimension of the transformer, this can be changed
    depth=6,  # Number of transformer layers, this can be changed
    heads=8,  # Number of attention heads, this can be changed
    dim_head=64,  # Dimension of each attention head, this can be changed
    mlp_dim=256,  # Dimension of the MLP layer, this can be changed
    num_classes=15,  # Number of classes, this can be changed
    dropout=0.4,  # Dropout rate, this can be changed
    pos_emb_size=128,  # Position embedding size, this can be changed
    class_emb_size=128,  # Class embedding size, this can be changed
    stride=1  # Stride for the convolution, this can be changed
)


In [ ]:
# Assume we have loaded one batch of HSI and LiDAR data
hsi_batch = hsi_train # Shape: (2832, 5, 5, 144)
lidar_batch = lidar_train # Shape: (2832, 5, 5, 1)
print('hsi_batch sahpe before transpose:', hsi_batch.shape)
print('lidar_batch sahpe before transpose:', lidar_batch.shape)
# Transpose the data to have the channel dimension at the correct place
hsi_batch = hsi_batch.transpose(0, 3, 1, 2)  # New shape: (2832, 144, 5, 5)
lidar_batch = lidar_batch.transpose(0, 3, 1, 2)  # New shape: (2832, 1, 5, 5)
print('hsi_batch sahpe after transpose:', hsi_batch.shape)
print('lidar_batch shape after transpose:', lidar_batch.shape)

hsi_batch sahpe before transpose: (16992, 9, 9, 144)
lidar_batch sahpe before transpose: (16992, 9, 9, 1)
hsi_batch sahpe after transpose: (16992, 144, 9, 9)
lidar_batch shape after transpose: (16992, 1, 9, 9)


In [ ]:
# Assume we have loaded one batch of HSI and LiDAR data
one_hsi_batch = hsi_train[:1]  # Shape: (1, 5, 5, 144)
one_lidar_batch = lidar_train[:1]  # Shape: (1, 5, 5, 1)
print('one_hsi_batch sahpe before transpose:', one_hsi_batch.shape)
print('one_lidar_batch sahpe before transpose:', one_lidar_batch.shape)
# Transpose the data to have the channel dimension at the correct place
one_hsi_batch = one_hsi_batch.transpose(0, 3, 1, 2)  # New shape: (1, 144, 5, 5)
one_lidar_batch = one_lidar_batch.transpose(0, 3, 1, 2)  # New shape: (1, 1, 5, 5)
print('hsi_batch sahpe after transpose:', one_hsi_batch.shape)
print('lidar_batch shape after transpose:', one_lidar_batch.shape)

one_hsi_batch sahpe before transpose: (1, 9, 9, 144)
one_lidar_batch sahpe before transpose: (1, 9, 9, 1)
hsi_batch sahpe after transpose: (1, 144, 9, 9)
lidar_batch shape after transpose: (1, 1, 9, 9)


In [ ]:
hsi_patch_embedding = PatchEmbedding(hsi_config).to(device)
lidar_patch_embedding = PatchEmbedding(lidar_config).to(device)

# Assume we have loaded one batch of HSI and LiDAR data
one_hsi_batch = hsi_train[:1]  # Shape: (1, 5, 5, 144)
one_lidar_batch = lidar_train[:1]  # Shape: (1, 5, 5, 1)

# Now reshape HSI data such that spatial dimensions (5x5) are flattened and treated as channels
one_hsi_batch_flat = torch.from_numpy(one_hsi_batch.astype(np.float32).reshape(1, hsi_config.in_channels, hsi_config.num_patches, 1)).to(device)
one_lidar_batch_flat = torch.from_numpy(one_lidar_batch.astype(np.float32).reshape(1, lidar_config.in_channels, lidar_config.num_patches, 1)).to(device)

# Initialize the patch embedding module
hsi_patch_embedding = PatchEmbedding(hsi_config).to(device)
lidar_patch_embedding = PatchEmbedding(lidar_config).to(device)

# Pass the data through the patch embedding module
one_hsi_batch_embedded = hsi_patch_embedding(one_hsi_batch_flat).to(device)
one_lidar_batch_embedded = lidar_patch_embedding(one_lidar_batch_flat).to(device)

print('one_hsi_batch_embedded shape:', one_hsi_batch_embedded.shape)
print('one_lidar_batch_embedded shape:', one_lidar_batch_embedded.shape)


one_hsi_batch_embedded shape: torch.Size([1, 145, 128])
one_lidar_batch_embedded shape: torch.Size([1, 2, 128])


### Intialisation CrossAttention Class

In [ ]:
# Assume we have loaded one batch of HSI and LiDAR data
hsi_batch_6 = hsi_train[:6]  # Shape: (6, 5, 5, 144)
lidar_batch_6 = lidar_train[:6]  # Shape: (6, 5, 5, 1)

print("hsi_batch_6  shape:", hsi_batch_6 .shape)
print("lidar_batch_6 shape:", lidar_batch_6.shape)

# Now reshape HSI data such that spatial dimensions (5x5) are flattened and treated as channels
hsi_batch_6_flat = torch.from_numpy(hsi_batch_6.astype(np.float32).reshape(6, hsi_config.in_channels, hsi_config.num_patches, 1))
lidar_batch_6_flat = torch.from_numpy(lidar_batch_6.astype(np.float32).reshape(6, lidar_config.in_channels, lidar_config.num_patches, 1))

print("hsi_batch_6_flat  shape:", hsi_batch_6_flat .shape)
print("lidar_batch_6_flat shape:", lidar_batch_6_flat.shape)

device = torch.device("cpu")

hsi_batch_6_flat = hsi_batch_6_flat.to(device)  # Move the tensor to GPU
lidar_batch_6_flat = lidar_batch_6_flat.to(device)  # Move the tensor to GPU


hsi_patch_embedding = PatchEmbedding(hsi_config).to(device)
lidar_patch_embedding = PatchEmbedding(lidar_config).to(device)

# Pass the data through the patch embedding module
hsi_batch_6_embedded  = hsi_patch_embedding(hsi_batch_6_flat)
lidar_batch_6_embedded = lidar_patch_embedding(lidar_batch_6_flat)

#device = torch.device("cuda:0")  # Define the device (GPU)
device = torch.device("cpu")
# Move the tensors to the desired device
hsi_batch_6_embedded = hsi_batch_6_embedded.to(device)
lidar_batch_6_embedded = lidar_batch_6_embedded.to(device)
print("hsi_batch_6_embedded  shape:", hsi_batch_6_embedded .shape)
print("lidar_batch_6_embedded shape:", lidar_batch_6_embedded.shape)

# Define the dimension of the model and the number of heads
d_model = hsi_config.emb_size  # the output dimension of PatchEmbedding
num_heads = hsi_config.heads  # the number of attention heads in the transformer

# Initialize CrossAttention module
cross_attention = CrossAttention(lidar_config, hsi_config).to(device)

# Apply the cross attention
output,attn_scores = cross_attention(lidar_batch_6_embedded, hsi_batch_6_embedded)


print("Cross attention output shape:", attn_scores.shape)
print("Output shape:", output.shape)




hsi_batch_6  shape: (6, 9, 9, 144)
lidar_batch_6 shape: (6, 9, 9, 1)
hsi_batch_6_flat  shape: torch.Size([6, 81, 144, 1])
lidar_batch_6_flat shape: torch.Size([6, 81, 1, 1])
hsi_batch_6_embedded  shape: torch.Size([6, 145, 128])
lidar_batch_6_embedded shape: torch.Size([6, 2, 128])
Cross attention output shape: torch.Size([6, 144, 144])
Output shape: torch.Size([6, 144, 512])


In [ ]:
cross_attention

CrossAttention(
  (to_q): Linear(in_features=128, out_features=512, bias=False)
  (to_k): Linear(in_features=128, out_features=512, bias=False)
  (to_v): Linear(in_features=128, out_features=512, bias=False)
  (to_out): Linear(in_features=512, out_features=144, bias=True)
)

In [ ]:
# Create model instance
model = CrossAttentionModel(hsi_config, lidar_config).to(device)
model

CrossAttentionModel(
  (hsi_patch_embedding): PatchEmbedding(
    (proj): Conv2d(81, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (lidar_patch_embedding): PatchEmbedding(
    (proj): Conv2d(81, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (cross_attention): CrossAttention(
    (to_q): Linear(in_features=128, out_features=512, bias=False)
    (to_k): Linear(in_features=128, out_features=512, bias=False)
    (to_v): Linear(in_features=128, out_features=512, bias=False)
    (to_out): Linear(in_features=512, out_features=144, bias=True)
  )
)

# 4.1  Training DataLoader for Cross Attention Module

In [ ]:
class HyperspectralDataset(Dataset):
    def __init__(self, hsi_samples, lidar_samples, labels):
        self.hsi_samples = hsi_samples
        self.lidar_samples = lidar_samples
        self.labels = labels

    def __len__(self):
        return len(self.hsi_samples)

    def __getitem__(self, idx):
        hsi_patch = self.hsi_samples[idx].float().to(device)
        lidar_patch = self.lidar_samples[idx].float().to(device)
        label = self.labels[idx]

        # Convert label to a tensor and reshape to match the model's output shape
        label = torch.tensor(label).repeat(hsi_patch.shape[1], 1).to(device)

        return hsi_patch, lidar_patch, label

In [ ]:
labels_tensor = torch.tensor(y_train)
#labels_tensor -= 1  # Shift label values to the range [0, C-1]
print('labels_tensor:',labels_tensor.shape)

unique_classes = torch.unique(labels_tensor)
print("Unique classes:", unique_classes)

labels_tensor: torch.Size([16992, 15])
Unique classes: tensor([0, 1], dtype=torch.int32)


In [ ]:
# Assuming labels_tensor is one-hot encoded or contains probabilities
labels_tensor = torch.argmax(labels_tensor, dim=1)
labels_tensor.shape

torch.Size([16992])

In [ ]:
y_train=labels_tensor
print('y_train shape:',y_train.shape)

y_train shape: torch.Size([16992])


In [ ]:

y_test_tensor= torch.tensor(y_test)
y_test_tensor -= 1  # Shift label values to the range [0, C-1]
print('y_test_tensor shape:',y_test_tensor.shape)

y_test_tensor shape: torch.Size([12197, 15])


In [ ]:
# Assuming labels_tensor is one-hot encoded or contains probabilities
y_test_tensor = torch.argmax(y_test_tensor, dim=1)
print('y_test_tensor shape:',y_test_tensor.shape)

y_test_tensor shape: torch.Size([12197])


In [ ]:
y_test= y_test_tensor

In [ ]:
unique_classes = torch.unique(labels_tensor)
print("Unique classes:", unique_classes)

Unique classes: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


In [ ]:
from sklearn.model_selection import train_test_split


## Reshape the data
hsi_batch_flat = torch.from_numpy(hsi_train.astype(np.float32).reshape(16992, hsi_config.in_channels, 144, 1))
lidar_batch_flat = torch.from_numpy(lidar_train.astype(np.float32).reshape(16992, lidar_config.in_channels, 1, 1))

print("hsi_batch shape:", hsi_batch_flat .shape)
print("lidar_batch shape:", lidar_batch_flat.shape)

hsi_batch_flat = hsi_batch_flat.to(device)  # Move the tensor to GPU
lidar_batch_flat = lidar_batch_flat.to(device)  # Move the tensor to GPU


# Split train into train and validation
hsi_samples_train, hsi_samples_val, lidar_samples_train, lidar_samples_val, labels_train, labels_val = train_test_split(
    hsi_batch_flat, lidar_batch_flat, y_train, test_size=0.20, random_state=42)
print('hsi_samples_train shape:',hsi_samples_train.shape)
print('hsi_samples_val shape:',hsi_samples_val.shape)
print('lidar_samples_train shape:',lidar_samples_train.shape)
print('lidar_samples_valshape:',lidar_samples_val.shape)
print('labels_train, shape:',labels_train.shape)
print('labels_val, shape:',labels_val.shape)

# Now test data.
## Reshape the data
hsi_test_batch_flat = torch.from_numpy(hsi_test.astype(np.float32).reshape(12197, hsi_config.in_channels, 144, 1))
lidar_test_batch_flat = torch.from_numpy(lidar_test.astype(np.float32).reshape(12197, lidar_config.in_channels, 1, 1))

# Create Datasets
train_dataset = HyperspectralDataset(hsi_samples_train, lidar_samples_train, labels_train)
val_dataset = HyperspectralDataset(hsi_samples_val, lidar_samples_val, labels_val)
test_dataset = HyperspectralDataset(hsi_test_batch_flat, lidar_test_batch_flat, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

hsi_batch shape: torch.Size([16992, 81, 144, 1])
lidar_batch shape: torch.Size([16992, 81, 1, 1])
hsi_samples_train shape: torch.Size([13593, 81, 144, 1])
hsi_samples_val shape: torch.Size([3399, 81, 144, 1])
lidar_samples_train shape: torch.Size([13593, 81, 1, 1])
lidar_samples_valshape: torch.Size([3399, 81, 1, 1])
labels_train, shape: torch.Size([13593])
labels_val, shape: torch.Size([3399])


# 4.2 Training CorssAttentionMode ltraining

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import copy

# Define the batch size
batch_size = 32

# Create the data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model instance
model = CrossAttentionModel(hsi_config, lidar_config).to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1.00) #loss 2.3866

# Initialize the best validation loss and best_model_wts before the training loop
best_val_loss = float('inf')  # Initialize with a very high value
best_model_wts = copy.deepcopy(model.state_dict())  # Initialize with the model weights

# Instantiate the loss function
criterion = nn.CrossEntropyLoss()

# Initialize the lowest_loss and best_model_wts before the training loop
lowest_loss = float('inf')  # Initialize with a very high value
best_model_wts = copy.deepcopy(model.state_dict())  # Initialize with the model weights

num_epochs = 200

for epoch in range(num_epochs):
    for batch_idx, (hsi_batch, lidar_batch, label_batch) in enumerate(train_loader):
        # Move tensors to the configured device
        hsi_batch = hsi_batch.to(device)
        lidar_batch = lidar_batch.to(device)
        label_batch = label_batch.to(device)  # Reshape labels
        # print('hsi_batch shape:', hsi_batch.shape)
        # print('lidar_batch:', lidar_batch.shape)
        # print('label_batch shape:', label_batch.shape)

        # Forward pass
        output,attn_scores  = model(lidar_batch, hsi_batch)
        #print('output shape:', output.shape)

        # Calculate loss
        loss = criterion(output.transpose(1, 2), label_batch.squeeze(2))

        # Print loss every 10 batches
        #if (batch_idx + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


        # If the loss is lower than the current lowest, save the model's state
        if loss.item() < lowest_loss:
            lowest_loss = loss.item()
            best_model_wts = copy.deepcopy(model.state_dict())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Step the learning rate scheduler
        scheduler.step()

# Print finishing training
print('Finishing training')

# Save the best model weights
#torch.save(best_model_wts, 'best_model_weights.pth')


<ipython-input-42-b31b03d9f721>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label).repeat(hsi_patch.shape[1], 1).to(device)


Epoch [1/200], Batch [1/425], Loss: 13.5947
Epoch [1/200], Batch [2/425], Loss: 13.3491
Epoch [1/200], Batch [3/425], Loss: 12.2305
Epoch [1/200], Batch [4/425], Loss: 12.9050
Epoch [1/200], Batch [5/425], Loss: 12.2903
Epoch [1/200], Batch [6/425], Loss: 11.7252
Epoch [1/200], Batch [7/425], Loss: 12.4860
Epoch [1/200], Batch [8/425], Loss: 11.6189
Epoch [1/200], Batch [9/425], Loss: 11.2262
Epoch [1/200], Batch [10/425], Loss: 10.9239
Epoch [1/200], Batch [11/425], Loss: 10.3285
Epoch [1/200], Batch [12/425], Loss: 10.1136
Epoch [1/200], Batch [13/425], Loss: 8.6381
Epoch [1/200], Batch [14/425], Loss: 9.1455
Epoch [1/200], Batch [15/425], Loss: 8.1612
Epoch [1/200], Batch [16/425], Loss: 8.7146
Epoch [1/200], Batch [17/425], Loss: 8.7865
Epoch [1/200], Batch [18/425], Loss: 8.1203
Epoch [1/200], Batch [19/425], Loss: 8.7570
Epoch [1/200], Batch [20/425], Loss: 8.0156
Epoch [1/200], Batch [21/425], Loss: 8.0290
Epoch [1/200], Batch [22/425], Loss: 7.4068
Epoch [1/200], Batch [23/425]

KeyboardInterrupt: 

In [ ]:
# Save the best model state
torch.save(best_model_wts, 'best_model_aug_p9_weights.pth')


In [ ]:
# Load the saved model
best_model = CrossAttentionModel(hsi_config, lidar_config)

# Load the weights
best_model.load_state_dict(torch.load('best_model_aug_p9_weights.pth'))

# Move the model to the GPU
best_model = best_model.to(device)

# Now your model is ready for making predictions on GPU


# 5.0 Load Trained Model

In [ ]:
# Load the model
model = CrossAttentionModel(hsi_config, lidar_config)
model.load_state_dict(torch.load('best_model_aug_p9_weights.pth'))
model.eval()  # set the model to evaluation mode

# Move the model to the GPU
#
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())


Model's state_dict:
hsi_patch_embedding.pos_embedding 	 torch.Size([1, 145, 128])
hsi_patch_embedding.cls_token 	 torch.Size([1, 1, 128])
hsi_patch_embedding.proj.weight 	 torch.Size([128, 81, 1, 1])
hsi_patch_embedding.proj.bias 	 torch.Size([128])
lidar_patch_embedding.pos_embedding 	 torch.Size([1, 2, 128])
lidar_patch_embedding.cls_token 	 torch.Size([1, 1, 128])
lidar_patch_embedding.proj.weight 	 torch.Size([128, 81, 1, 1])
lidar_patch_embedding.proj.bias 	 torch.Size([128])
cross_attention.to_q.weight 	 torch.Size([512, 128])
cross_attention.to_k.weight 	 torch.Size([512, 128])
cross_attention.to_v.weight 	 torch.Size([512, 128])
cross_attention.to_out.weight 	 torch.Size([144, 512])
cross_attention.to_out.bias 	 torch.Size([144])


In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor])


Model's state_dict:
hsi_patch_embedding.pos_embedding 	 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
hsi_patch_embedding.cls_token 	 tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])
hsi_patch_embedding.proj.weight 	 tensor([[[[-0.0568]],

         [[-0.0421

In [ ]:
# Ensure the model is in evaluation mode
best_model.eval()

# Initialize a list to hold all the predictions and attention scores
all_predictions = []
all_attn_scores = []

# Loop over the validation set
for hsi_batch, lidar_batch, _ in val_loader:  # we don't need the labels for predictions
    # Move the batch to the desired device
    hsi_batch = hsi_batch.to(device)
    lidar_batch = lidar_batch.to(device)

    # Pass the batch through the model
    with torch.no_grad():
        output, attn_scores = best_model(lidar_batch, hsi_batch)

        # Add the predictions and attention scores to our lists
        all_predictions.append(output.cpu().numpy())
        all_attn_scores.append(attn_scores.cpu().numpy())

# Concatenate all predictions and attention scores into a single numpy array
all_predictions = np.concatenate(all_predictions)
all_attn_scores = np.concatenate(all_attn_scores)

# Now you can use the predictions and attention scores as needed
print('all_predictions shape:', all_predictions.shape )
print('all_attn_scores shape:', all_attn_scores.shape )


<ipython-input-39-b31b03d9f721>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label).repeat(hsi_patch.shape[1], 1).to(device)


all_predictions shape: (3399, 144, 512)
all_attn_scores shape: (3399, 144, 144)


# Band Selection

Calculate the variance of the feature vectors for each band across all samples and then find the bands with the maximum variance

In [ ]:
# First, take the mean across all samples and heads. This will result in a single 144-dim vector.
mean_attention = np.mean(all_attn_scores, axis=(0, 1))
print('mean_attention:',mean_attention.shape)

# Then, sort the bands by attention. This will give you the band indices in descending order of attention.
sorted_band_indices = np.argsort(mean_attention)[::-1]
print('sorted_band_indices :',sorted_band_indices.shape)

# If you want to see the attention values as well, you can sort the mean_attention array in the same order.
sorted_attention_values = mean_attention[sorted_band_indices]
print('sorted_attention_values :',sorted_attention_values.shape)

N = 50 #op N bands
top_N_bands = sorted_band_indices[:N]
top_N_attention_values = sorted_attention_values[:N]

# Print the top N band indices with their attention scores
print(f"Top {N} bands based on the mean attention score:")
for band_index, attention_value in zip(top_N_bands, top_N_attention_values):
    print(f"Band index: {band_index}, Mean attention score: {attention_value}")

# Print the list of top N band indices
print("\nList of top {} band indices:".format(N))
print(top_N_bands.tolist())


mean_attention: (144,)
sorted_band_indices : (144,)
sorted_attention_values : (144,)
Top 50 bands based on the mean attention score:
Band index: 41, Mean attention score: 32.04867935180664
Band index: 87, Mean attention score: 30.620180130004883
Band index: 75, Mean attention score: 30.55144500732422
Band index: 90, Mean attention score: 29.045427322387695
Band index: 54, Mean attention score: 27.903484344482422
Band index: 20, Mean attention score: 25.40079689025879
Band index: 119, Mean attention score: 23.460657119750977
Band index: 58, Mean attention score: 21.71855354309082
Band index: 138, Mean attention score: 21.27195167541504
Band index: 1, Mean attention score: 20.67475128173828
Band index: 33, Mean attention score: 20.39455223083496
Band index: 126, Mean attention score: 20.15049171447754
Band index: 129, Mean attention score: 19.294536590576172
Band index: 38, Mean attention score: 18.89678955078125
Band index: 82, Mean attention score: 18.88149642944336
Band index: 84, Mea

In [ ]:
bands_aug=[41, 87, 75, 90, 54, 20, 119, 58, 138, 1, 33, 126, 129, 38, 82, 84, 107, 133, 14, 88, 59, 68, 125, 15, 17, 57, 110, 24, 74, 141, 96, 46, 95, 11, 128, 27, 108, 28, 140, 32, 65, 13, 10, 25, 111, 30, 93, 77, 39, 106]

In [ ]:
#Patch_sixe=1
bands_p1=[73, 100, 80, 101, 26, 40, 78, 121, 136, 107, 50, 122, 63, 113, 12, 141, 8, 139, 81, 127, 17, 116, 19, 125, 33, 128, 97, 36, 79, 142, 6, 134, 108, 55, 89, 75, 14, 46, 5, 72, 10, 35, 135, 96, 2, 130, 64, 129, 18, 37]

In [ ]:
band_50=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101]
band_45=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21]
band_40=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56]
band_35=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58]
band_30=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85]
band_25=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81]
band_20=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48]
band_15=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75]
band_10=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139]
band_5=[115, 127, 63, 113, 54]
band_80=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101, 24, 73, 32, 141, 72, 131, 70, 49, 71, 65, 66, 45, 110, 93, 22, 16, 14, 123, 107, 100]
band_75=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101, 24, 73, 32, 141, 72, 131, 70, 49, 71, 65, 66, 45, 110, 93, 22]
band_70=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101, 24, 73, 32, 141, 72, 131, 70, 49, 71, 65]
band_65=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101, 24, 73, 32, 141, 72]
band_60=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101]
band_55=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21]

In [ ]:
bands_p3=[54, 49, 21, 142, 83, 20, 16, 15, 109, 112, 132, 100, 135, 113, 42, 23, 93, 130, 9, 104, 5, 76, 91, 14, 44, 26, 129, 99, 120, 77, 137, 29, 111, 46, 70, 38, 101, 75, 84, 48, 102, 60, 27, 126, 56, 61, 97, 78, 6, 36, 2, 87, 64, 95, 143, 68, 34, 69, 71, 79, 131, 28, 82, 96, 66, 58, 12, 106, 35, 40, 114, 17, 116, 57, 105, 88, 33, 107, 37, 136, 1, 51, 85, 62, 0, 8, 141, 32, 65, 92, 121, 89, 50, 138, 4, 22, 103, 39, 90, 127, 59, 98, 139, 53, 63, 119, 73, 43, 52, 118, 117, 110, 24, 125, 11, 123, 19, 86, 7, 74, 115, 67, 55, 80, 3, 41, 140, 31, 128, 124, 45, 122, 72, 10, 13, 108, 25, 133, 47, 81, 134, 30, 18, 94]

In [ ]:
# patch_size=5
bands_p5=[115, 127, 63, 113, 54, 12, 60, 62, 4, 139, 50, 90, 51, 112, 75, 42, 11, 97, 106, 48, 1, 35, 109, 76, 81, 53, 124, 80, 137, 85, 125, 37, 105, 135, 58, 126, 7, 40, 15, 56, 19, 18, 3, 38, 21, 61, 138, 95, 118, 101, 24, 73, 32, 141, 72, 131, 70, 49, 71, 65, 66, 45, 110, 93, 22, 16, 14, 123, 107, 100, 26, 6, 103, 79, 29, 92, 69, 5, 111, 128, 74, 31, 130, 102, 64, 9, 134, 67, 44, 34, 91, 89, 104, 121, 77, 86, 68, 59, 84, 114, 83, 119, 120, 43, 88, 55, 17, 133, 13, 116, 10, 98, 25, 39, 87, 23, 78, 27, 82, 96, 20, 108, 136, 140, 8, 28, 0, 132, 99, 94, 52, 47, 2, 142, 33, 57, 122, 143, 46, 30, 36, 117, 129, 41]

In [ ]:
#Patch_size=7
band_p7=[130, 3, 85, 23, 40, 82, 119, 127, 95, 131, 108, 47, 36, 63, 67, 114, 141, 99, 59, 10, 58, 84, 5, 73, 9, 50, 0, 35, 7, 20, 89, 129, 24, 57, 126, 93, 134, 55, 14, 103, 140, 38, 139, 21, 135, 48, 74, 115, 66, 136, 100, 52, 65, 18, 45, 56, 76, 111, 61, 107, 113, 68, 8, 17, 16, 44, 91, 81, 51, 27, 138, 53, 118, 109, 143, 1, 122, 120, 11, 70, 32, 69, 125, 62, 90, 33, 28, 37, 102, 34, 6, 12, 80, 77, 79, 49, 105, 72, 124, 83, 19, 54, 15, 117, 31, 43, 39, 106, 96, 98, 116, 112, 133, 71, 92, 86, 128, 78, 101, 26, 94, 87, 64, 22, 13, 30, 42, 25, 75, 123, 29, 41, 142, 121, 132, 137, 4, 110, 2, 104, 97, 60, 46, 88]

In [ ]:
#Patch_size=9
bands_p9=[42, 131, 71, 17, 56, 114, 47, 53, 124, 52, 139, 135, 50, 16, 48, 8, 11, 73, 99, 2, 63, 28, 88, 116, 32, 14, 133, 57, 102, 137, 15, 79, 132, 105, 120, 93, 80, 43, 81, 138, 142, 31, 97, 130, 34, 26, 110, 27, 134, 69, 39, 78, 37, 106, 77, 25, 29, 89, 95, 65, 141, 129, 18, 62, 91, 121, 58, 127, 118, 1, 19, 36, 30, 20, 128, 44, 104, 24, 107, 117, 0, 84, 21, 112, 4, 68, 125, 5, 103, 45, 33, 70, 111, 109, 23, 94, 136, 64, 12, 143, 85, 72, 122, 119, 83, 82, 38, 108, 66, 100, 55, 35, 40, 49, 101, 51, 3, 59, 9, 54, 41, 140, 86, 7, 10, 75, 46, 22, 98, 87, 13, 92, 90, 76, 61, 60, 115, 113, 123, 96, 126, 67, 6, 74]

In [ ]:
#Patch_size=11
bands_p11=[78, 84, 41, 49, 131, 5, 59, 118, 79, 136, 40, 86, 69, 104, 88, 100, 45, 26, 7, 81, 3, 99, 92, 132, 22, 119, 9, 127, 56, 109, 102, 19, 44, 116, 4, 107, 85, 53, 17, 113, 24, 65, 133, 54, 101, 141, 106, 13, 143, 63, 114, 108, 89, 27, 8, 57, 137, 58, 111, 47, 25, 42, 68, 48, 98, 38, 130, 36, 73, 82, 6, 95, 77, 115, 15, 71, 60, 21, 11, 110, 122, 117, 1, 123, 134, 52, 126, 0, 125, 55, 97, 43, 76, 120, 39, 18, 142, 74, 16, 67, 46, 2, 83, 37, 61, 135, 112, 31, 12, 10, 91, 129, 94, 80, 72, 64, 103, 34, 23, 90, 66, 121, 70, 29, 140, 14, 96, 138, 62, 32, 30, 139, 35, 93, 50, 87, 33, 51, 128, 75, 124, 105, 20, 28]

In [ ]:
#Patch_size=13
bands_p13=[143, 99, 109, 13, 16, 33, 94, 97, 53, 68, 89, 77, 93, 47, 87, 19, 2, 136, 57, 104, 26, 132, 38, 11, 76, 40, 140, 111, 119, 96, 102, 108, 90, 131, 113, 49, 62, 58, 56, 35, 128, 18, 14, 51, 110, 66, 24, 117, 43, 71, 137, 81, 74, 141, 120, 135, 60, 72, 103, 9, 21, 8, 3, 107, 123, 6, 37, 129, 138, 65, 44, 116, 78, 22, 10, 39, 73, 23, 121, 28, 86, 88, 139, 75, 31, 46, 27, 92, 115, 25, 114, 30, 91, 20, 48, 98, 50, 59, 17, 41, 79, 12, 101, 133, 105, 29, 69, 63, 1, 142, 15, 54, 126, 42, 7, 124, 85, 106, 125, 4, 0, 82, 80, 36, 5, 34, 67, 118, 70, 55, 64, 84, 52, 112, 61, 83, 32, 122, 95, 45, 127, 100, 130, 134]

In [ ]:
#Patch_size=15
bands_p15=[138, 102, 136, 112, 130, 135, 85, 75, 97, 27, 71, 11, 38, 51, 29, 16, 47, 93, 39, 73, 9, 59, 31, 20, 30, 70, 5, 14, 65, 23, 61, 86, 119, 108, 56, 101, 33, 105, 116, 120, 43, 91, 74, 88, 52, 82, 4, 79, 19, 55, 21, 60, 36, 78, 45, 44, 28, 22, 37, 53, 7, 140, 90, 32, 100, 96, 48, 92, 141, 104, 68, 80, 98, 34, 94, 114, 58, 62, 35, 26, 106, 83, 41, 110, 127, 109, 72, 76, 13, 64, 103, 132, 99, 69, 54, 122, 12, 129, 89, 126, 63, 50, 49, 125, 124, 10, 66, 113, 2, 84, 121, 77, 18, 67, 17, 133, 8, 46, 134, 143, 115, 81, 42, 142, 118, 131, 25, 15, 57, 123, 139, 137, 6, 24, 3, 117, 1, 40, 95, 111, 87, 107, 128, 0]